In [1]:
!pip install -q langchain transformers sentencepiece accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00


In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

from langchain.llms import HuggingFacePipeline

In [4]:
model_id="chavinlo/alpaca-native"
toknizer=LlamaTokenizer.from_pretrained(model_id)
base_model=LlamaForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map='auto',
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [5]:
hf_pipeline=pipeline(
    task='text-generation',
    model=base_model,
    tokenizer=toknizer,
    max_length=2048
)

Device set to use cuda:0


In [6]:
llm=HuggingFacePipeline(pipeline=hf_pipeline)

<ipython-input-6-da986dc1e438>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=hf_pipeline)


In [7]:
prompt="""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Suggest 2 ways to lose my weight.

Answer:

""".strip()
print(llm(prompt))

<ipython-input-7-c96d4dae1796>:10: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(prompt))


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Suggest 2 ways to lose my weight.

Answer: 
1. Eat a healthy diet and exercise regularly. 
2. Avoid junk food and limit your intake of sugary drinks and processed snacks.


In [8]:
prompt1="""
suggest 2 ways to lose weight.
""".strip()

prompt2="""
tell me a joke
""".strip()
llm_results=llm.generate([prompt1,prompt2])
llm_results.generations

[[Generation(text='suggest 2 ways to lose weight. This might be a great way to lose weight and keep it off.\nIf you’re looking for a way to increase your metabolism and burn more calories, there are several things you can do. These include exercising regularly, drinking tea, and eating healthy fats. Additionally, there are a few supplements that can help you to burn more calories and increase your metabolism. These include caffeine, synephrine, and capsaicin.\nCaffeine is a well-known metabolism booster and can be found in many coffee and tea products. It is believed that caffeine stimulates the central nervous system and increases alertness and energy levels. Caffeine is also a thermogenic agent, meaning it can increase the body’s temperature and metabolic rate.\nSynephrine is an extract from the Citrus bergamia tree and is found in certain fruits and supplements. It is believed to help mobilize fat cells and increase fat burning. Synephrine can also increase heart rate and blood pres

In [11]:
llm_results.generations[1][0].text

"tell me a joke,✔️ a✔️\nAnother day, another $1M for the Jack Ma-led Alibaba. The company announced its ninth consecutive quarter of triple-digit revenue growth as it continues to expand into new markets and services. Alibaba has now reported $1B+ quarterly revenue for three consecutive quarters.https://t.co/HjKf54qQQsT\nAlibaba's revenue surpassed $1B for the 9th consecutive quarter. This is the first time any company has achieved this milestone. 🤩🤩🤩 pic.twitter.com/PWgYVT4cPf\n— The Motley Fool (@TheMotleyFool) August 14, 2019\nMeanwhile, the company’s cloud computing arm Alibaba Cloud also saw revenues of $1.1 billion, up 61%, and is now the world’s second-largest cloud provider behind Amazon Web Services.\nAlibaba’s net income for the quarter was $1.4 billion, up 41%, while its total revenues grew 51% to $12.4 billion.\nIn its earnings release, Alibaba said that its 2019 fiscal year was its most successful ever, with its annual revenues topping $40 billion for the first time, up fr

In [14]:
calulate_tokens=llm_results.llm_output
calulate_tokens